In [ ]:
pip install rasterio

In [ ]:
# To suppress all warnings

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from absl import logging
logging.set_verbosity(logging.ERROR)

import tensorflow as tf

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="keras")

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # hide TF logs

import tensorflow as tf
from tensorflow import keras

import os
os.environ['XLA_FLAGS'] = '--xla_cpu_enable_fast_math=false'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
from rasterio.errors import NotGeoreferencedWarning

warnings.filterwarnings("ignore", category=NotGeoreferencedWarning)




In [ ]:
import os
import numpy as np
import rasterio
from skimage.transform import resize
import matplotlib.pyplot as plt

DATA_DIR = "/kaggle/input/satellite-images"
TRAIN_DIR = ("/kaggle/input/dataset11/train")
TEST_DIR = ("/kaggle/input/dataset11/test")

In [ ]:
import os

print("Train files:", os.listdir(TRAIN_DIR))
print("Test files:", os.listdir(TEST_DIR))


In [ ]:
# Path to an image
image_path = os.path.join(TRAIN_DIR, "0_image.tif")

# Open the image and inspect its metadata
with rasterio.open(image_path) as src:
    print("Image Metadata:")
    print(src.meta)
    print(f"Number of Bands: {src.count}")
    print(f"Data Type: {src.dtypes}")
    print(f"Shape: {src.height} x {src.width}")
    print(f"CRS: {src.crs}")


mask_path = os.path.join(TRAIN_DIR, "0_label.tif")
with rasterio.open(mask_path) as src:
    print("\nMask Metadata:")
    print(src.meta)
    print(f"Number of Bands: {src.count}")
    print(f"Data Type: {src.dtypes}")
    print(f"Shape: {src.height} x {src.width}")
    print(f"CRS: {src.crs}")

In [ ]:
def load_data(image_path, mask_path):
    # Load image
    with rasterio.open(image_path) as src:
        image = src.read()  # Read all bands
        image = np.transpose(image, (1, 2, 0))  # Reorder dimensions to (height, width, channels)


        if image.shape[2] == 4:  
            image = image[..., :3]

    # Load mask
    with rasterio.open(mask_path) as src:
        mask = src.read(1)  # Read single band (binary mask)

    return image, mask

def preprocess(image, mask, target_size=(256, 256)):
    # Resize image and mask
    image = resize(image, target_size + (image.shape[2],), anti_aliasing=True)
    mask = resize(mask, target_size, anti_aliasing=False)

    # Normalize image
    image_min = image.min()
    image_max = image.max()
    image = (image - image_min) / (image_max - image_min + 1e-8)


    mask = (mask > 0.5).astype(np.float32)

    return image, mask

def get_image_mask_paths(directory):
    image_paths = []
    mask_paths = []

    for filename in os.listdir(directory):
        if filename.endswith("_image.tif"):
            image_paths.append(os.path.join(directory, filename))
        elif filename.endswith("_label.tif"):
            mask_paths.append(os.path.join(directory, filename))


    image_paths.sort()
    mask_paths.sort()

    return image_paths, mask_paths

# Get paths for train and test datasets
train_image_paths, train_mask_paths = get_image_mask_paths(TRAIN_DIR)
test_image_paths, test_mask_paths = get_image_mask_paths(TEST_DIR)



In [ ]:
print(f"Train Images: {len(train_image_paths)}")
print(f"Train Masks: {len(train_mask_paths)}")
print(f"Test Images: {len(test_image_paths)}")
print(f"Test Masks: {len(test_mask_paths)}")

In [ ]:
sample_image_path = train_image_paths[0]
sample_mask_path = train_mask_paths[0]
image, mask = load_data(sample_image_path, sample_mask_path)

# Display raw data
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title("Raw Image")
plt.imshow(image)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("Raw Mask")
plt.imshow(mask, cmap='gray')
plt.axis('off')

plt.show()

In [ ]:
# Preprocess the sample image and mask
preprocessed_image, preprocessed_mask = preprocess(image, mask, target_size=(256, 256))

# Display preprocessed data
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title("Preprocessed Image")
plt.imshow(preprocessed_image)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("Preprocessed Mask")
plt.imshow(preprocessed_mask, cmap='gray')
plt.axis('off')

plt.show()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose
from tensorflow.keras.optimizers import Adam

def build_unet(input_shape=(256, 256, 3)):
    inputs = Input(input_shape)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    # Decoder
    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Build the U-Net model
input_shape = (256, 256, 3)
model = build_unet(input_shape)
model.summary()

# Load and preprocess all training data
X_train = np.array([preprocess(rasterio.open(path).read().transpose((1, 2, 0)),
                              rasterio.open(train_mask_paths[i]).read(1),
                              target_size=(256, 256))[0]
                   for i, path in enumerate(train_image_paths)])
Y_train = np.array([preprocess(rasterio.open(path).read().transpose((1, 2, 0)),
                              rasterio.open(train_mask_paths[i]).read(1),
                              target_size=(256, 256))[1]
                   for i, path in enumerate(train_image_paths)])

# Load and preprocess all test data
X_test = np.array([preprocess(rasterio.open(path).read().transpose((1, 2, 0)),
                             rasterio.open(test_mask_paths[i]).read(1),
                             target_size=(256, 256))[0]
                  for i, path in enumerate(test_image_paths)])
Y_test = np.array([preprocess(rasterio.open(path).read().transpose((1, 2, 0)),
                             rasterio.open(test_mask_paths[i]).read(1),
                             target_size=(256, 256))[1]
                  for i, path in enumerate(test_image_paths)])

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, Y_train, epochs=100, batch_size=4, validation_split=0.2)

In [ ]:
# Predict on test data
predictions = model.predict(X_test)

# Display predictions
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.title("Satellite Image")
plt.imshow(X_test[0])
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("Ground Truth Mask")
plt.imshow(Y_test[0], cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title("Predicted Mask")
plt.imshow(predictions[0, :, :, 0], cmap='gray')
plt.axis('off')

plt.show()

In [ ]:
model.save("building_footprint_model.h5")

In [ ]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("building_footprint_model.h5")

In [ ]:
import numpy as np
from skimage.transform import resize
import rasterio

def preprocess(image_path, target_size=(256, 256)):
    # Load image
    with rasterio.open(image_path) as src:
        image = src.read()  # Read all bands
        image = np.transpose(image, (1, 2, 0))  # Reorder dimensions to (height, width, channels)

        # Remove alpha channel if present
        if image.shape[2] == 4:  # RGBA -> RGB
            image = image[..., :3]

    # Resize image
    image = resize(image, target_size + (image.shape[2],), anti_aliasing=True)

    # Normalize image
    image_min = image.min()
    image_max = image.max()
    image = (image - image_min) / (image_max - image_min + 1e-8)  # Avoid division by zero

    return image

# Get test image paths
test_image_paths = get_image_mask_paths(TEST_DIR)[0]

# Preprocess test images
X_test = np.array([preprocess(path, target_size=(256, 256)) for path in test_image_paths])

from sklearn.metrics import jaccard_score

y_true = Y_test.flatten()
y_pred = (predictions > 0.5).astype(np.uint8).flatten()

iou = jaccard_score(y_true, y_pred, average='micro')
print(f"Mean IoU: {iou}")

# Predict building footprints
predictions = model.predict(X_test)

# Ensure predictions are binary masks
predicted_masks = (predictions > 0.5).astype(np.float32)

In [ ]:
import matplotlib.pyplot as plt

# Display predictions
for i in range(len(X_test)):
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 3, 1)
    plt.title("Satellite Image")
    plt.imshow(X_test[i])
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title("Ground Truth Mask")
    plt.imshow(rasterio.open(test_mask_paths[i]).read(1), cmap='gray')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title("Predicted Mask")
    plt.imshow(predicted_masks[i, :, :, 0], cmap='gray')
    plt.axis('off')

    plt.show()

In [ ]:
from sklearn.metrics import jaccard_score

# Flatten the masks
y_true = Y_test.flatten()
y_pred = predicted_masks.flatten()

iou = jaccard_score(y_true, y_pred, average='micro')
print(f"Mean IoU: {iou}")



In [ ]:
import rasterio
from rasterio.transform import from_bounds
import numpy as np

# Define bounds and transform for saving masks
bounds = (0, 0, 256, 256)  # Example bounds
transform = from_bounds(*bounds, 256, 256)

# Save predicted masks
for i, pred_mask in enumerate(predicted_masks):
    output_path = f"predicted_mask_{i}.tif"

    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=pred_mask.shape[0],
        width=pred_mask.shape[1],
        count=1,
        dtype=rasterio.float32,
        crs='+proj=latlong',
        transform=transform,
    ) as dst:
        dst.write(pred_mask.squeeze(), 1)

# Predict building footprints
predictions = model.predict(X_test)

# Convert predictions to binary masks (1 for buildings, 0 for background)
predicted_masks = (predictions > 0.5).astype(np.uint8)

In [ ]:
import rasterio
from rasterio.transform import from_bounds

# Define bounds and transform for saving masks
bounds = (0, 0, 256, 256)  # Example bounds
transform = from_bounds(*bounds, 256, 256)

# Save predicted masks as .tif files
for i, pred_mask in enumerate(predicted_masks):
    output_path = f"predicted_footprint_{i}.tif"

    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=pred_mask.shape[0],
        width=pred_mask.shape[1],
        count=1,
        dtype=rasterio.uint8,
        crs='+proj=latlong',
        transform=transform,
    ) as dst:
        dst.write(pred_mask.squeeze(), 1)


import matplotlib.pyplot as plt

# Display predictions
for i in range(len(X_test)):
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 3, 1)
    plt.title("Satellite Image")
    plt.imshow(X_test[i])
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title("Ground Truth Mask")
    plt.imshow(rasterio.open(test_mask_paths[i]).read(1), cmap='gray')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title("Predicted Footprint")
    plt.imshow(predicted_masks[i, :, :, 0], cmap='gray')
    plt.axis('off')

    plt.show()

In [ ]:
import numpy as np
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape

# Function to convert binary mask to vectorized polygons
def mask_to_polygons(mask, transform):
    results = (
        {'properties': {'class': v}, 'geometry': s}
        for i, (s, v) in enumerate(shapes(mask, mask=None, transform=transform))
    )
    geoms = list(results)
    return geoms

# Convert predicted masks to polygons
vectorized_footprints = []
for i, pred_mask in enumerate(predicted_masks):
    transform = from_bounds(0, 0, 256, 256, 256, 256)  # Adjust bounds as needed
    geoms = mask_to_polygons(pred_mask.squeeze(), transform)

    # Create a GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(geoms)
    gdf.crs = "EPSG:4326"  # Set coordinate reference system (adjust if needed)

    # Save as GeoJSON
    gdf.to_file(f"footprint_vector_{i}.geojson", driver="GeoJSON")
    vectorized_footprints.append(gdf)

import cv2
import numpy as np
import matplotlib.pyplot as plt

# Example predicted mask (binary)
predicted_mask = predicted_masks[0, :, :, 0]  # Assuming predicted_masks is already defined

# Convert the predicted mask to uint8 format (required by OpenCV)
predicted_mask = (predicted_mask * 255).astype(np.uint8)

# Find contours in the binary mask
contours, _ = cv2.findContours(predicted_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a blank image to draw the contours
boundary_image = np.zeros_like(predicted_mask)

# Draw the contours on the blank image
cv2.drawContours(boundary_image, contours, -1, 255, thickness=1)  # Thickness=1 ensures thin boundaries

# Display the original image, predicted mask, and extracted boundaries
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.title("Satellite Image")
plt.imshow(X_test[0])
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("Predicted Mask")
plt.imshow(predicted_mask, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title("Extracted Footprints (Boundaries)")
plt.imshow(boundary_image, cmap='gray')
plt.axis('off')

plt.show()

In [ ]:
import rasterio

# Load the ground truth mask for the first test image
ground_truth_mask = rasterio.open(test_mask_paths[0]).read(1)  # Read single band

import cv2
import numpy as np

# Convert the ground truth mask to uint8 format
ground_truth_mask_uint8 = (ground_truth_mask * 255).astype(np.uint8)

# Find contours in the ground truth mask
gt_contours, _ = cv2.findContours(ground_truth_mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a blank image to draw the ground truth boundaries
gt_boundary_image = np.zeros_like(ground_truth_mask_uint8)
cv2.drawContours(gt_boundary_image, gt_contours, -1, 255, thickness=1)  # Thickness=1 ensures thin boundaries

In [ ]:
import matplotlib.pyplot as plt

# Display all four components
plt.figure(figsize=(20, 5))

# Satellite Image
plt.subplot(1, 4, 1)
plt.title("Satellite Image")
plt.imshow(X_test[0])  # Assuming X_test contains the preprocessed satellite images
plt.axis('off')

# Ground Truth Mask
plt.subplot(1, 4, 2)
plt.title("Ground Truth Mask")
plt.imshow(ground_truth_mask, cmap='gray')
plt.axis('off')

# Predicted Mask
plt.subplot(1, 4, 3)
plt.title("Predicted Mask")
plt.imshow(predicted_mask, cmap='gray')  # Assuming predicted_mask is already defined
plt.axis('off')

# Extracted Footprints (Boundaries)
plt.subplot(1, 4, 4)
plt.title("Extracted Footprints (Boundaries)")
plt.imshow(boundary_image, cmap='gray')  # Assuming boundary_image is already defined
plt.axis('off')

plt.show()

In [ ]:
import cv2

# Save the ground truth mask
cv2.imwrite("ground_truth_mask.tif", ground_truth_mask_uint8)

# Save the predicted mask
cv2.imwrite("predicted_mask.tif", predicted_mask)

# Save the extracted footprints (boundaries)
cv2.imwrite("extracted_footprints.tif", boundary_image)

import rasterio
import numpy as np

# Load the satellite image
satellite_image = X_test[0]  # Assuming X_test contains the preprocessed satellite images

# Load the ground truth mask for the first test image
ground_truth_mask = rasterio.open(test_mask_paths[0]).read(1)  # Read single band

# Load the predicted mask for the first test image
predicted_mask = predicted_masks[0, :, :, 0]  # Assuming predicted_masks is already defined


import cv2

# Convert masks to uint8 format for OpenCV
ground_truth_mask_uint8 = (ground_truth_mask * 255).astype(np.uint8)
predicted_mask_uint8 = (predicted_mask * 255).astype(np.uint8)

# Find contours in the ground truth mask
gt_contours, _ = cv2.findContours(ground_truth_mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a blank image to draw the ground truth boundaries
gt_boundary_image = np.zeros_like(ground_truth_mask_uint8)
cv2.drawContours(gt_boundary_image, gt_contours, -1, 255, thickness=1)  # Thickness=1 ensures thin boundaries

# Find contours in the predicted mask
pred_contours, _ = cv2.findContours(predicted_mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a blank image to draw the predicted boundaries
pred_boundary_image = np.zeros_like(predicted_mask_uint8)
cv2.drawContours(pred_boundary_image, pred_contours, -1, 255, thickness=1)  # Thickness=1 ensures thin boundaries

In [ ]:
import matplotlib.pyplot as plt

# Display all five components
plt.figure(figsize=(20, 5))

# Satellite Image
plt.subplot(1, 5, 1)
plt.title("Satellite Image")
plt.imshow(satellite_image)
plt.axis('off')

# Ground Truth Mask
plt.subplot(1, 5, 2)
plt.title("Ground Truth Mask")
plt.imshow(ground_truth_mask, cmap='gray')
plt.axis('off')

# Predicted Mask
plt.subplot(1, 5, 3)
plt.title("Predicted Mask")
plt.imshow(predicted_mask, cmap='gray')
plt.axis('off')

# Ground Truth Footprints
plt.subplot(1, 5, 4)
plt.title("Ground Truth Footprints")
plt.imshow(gt_boundary_image, cmap='gray')
plt.axis('off')

# Predicted Footprints
plt.subplot(1, 5, 5)
plt.title("Predicted Footprints")
plt.imshow(pred_boundary_image, cmap='gray')
plt.axis('off')

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Display all test images
for i in range(len(X_test)):
    # Get the current test image and its corresponding masks
    satellite_image = X_test[i]
    ground_truth_mask = rasterio.open(test_mask_paths[i]).read(1)
    predicted_mask = predicted_masks[i, :, :, 0]

    # Extract contours for ground truth and predicted masks
    gt_contours, _ = cv2.findContours((ground_truth_mask * 255).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    pred_contours, _ = cv2.findContours((predicted_mask * 255).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create blank images to draw contours
    gt_boundary_image = np.zeros_like(ground_truth_mask)
    pred_boundary_image = np.zeros_like(predicted_mask)

    # Draw contours on blank images
    cv2.drawContours(gt_boundary_image, gt_contours, -1, 255, thickness=1)
    cv2.drawContours(pred_boundary_image, pred_contours, -1, 255, thickness=1)

    # Display all five components for the current image
    plt.figure(figsize=(20, 5))

    # Satellite Image
    plt.subplot(1, 5, 1)
    plt.title("Satellite Image")
    plt.imshow(satellite_image)
    plt.axis('off')

    # Ground Truth Mask
    plt.subplot(1, 5, 2)
    plt.title("Ground Truth Mask")
    plt.imshow(ground_truth_mask, cmap='gray')
    plt.axis('off')

    # Predicted Mask
    plt.subplot(1, 5, 3)
    plt.title("Predicted Mask")
    plt.imshow(predicted_mask, cmap='gray')
    plt.axis('off')

    # Ground Truth Footprints
    plt.subplot(1, 5, 4)
    plt.title("Ground Truth Footprints")
    plt.imshow(gt_boundary_image, cmap='gray')
    plt.axis('off')

    # Predicted Footprints
    plt.subplot(1, 5, 5)
    plt.title("Predicted Footprints")
    plt.imshow(pred_boundary_image, cmap='gray')
    plt.axis('off')

    plt.show()

In [ ]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
import cv2
import rasterio
from rasterio.transform import from_bounds

# Suppress the warning
warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)

# Display all test images
for i in range(len(X_test)):
    # Get the current test image and its corresponding masks
    satellite_image = X_test[i]
    ground_truth_mask = rasterio.open(test_mask_paths[i]).read(1)
    predicted_mask = predicted_masks[i, :, :, 0]

    # Extract contours for ground truth and predicted masks
    gt_contours, _ = cv2.findContours((ground_truth_mask * 255).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    pred_contours, _ = cv2.findContours((predicted_mask * 255).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create blank images to draw contours
    gt_boundary_image = np.zeros_like(ground_truth_mask)
    pred_boundary_image = np.zeros_like(predicted_mask)

    # Draw contours on blank images
    cv2.drawContours(gt_boundary_image, gt_contours, -1, 255, thickness=1)
    cv2.drawContours(pred_boundary_image, pred_contours, -1, 255, thickness=1)

    # Display all five components for the current image
    plt.figure(figsize=(20, 5))

    # Satellite Image
    plt.subplot(1, 5, 1)
    plt.title("Satellite Image")
    plt.imshow(satellite_image)
    plt.axis('off')

    # Ground Truth Mask
    plt.subplot(1, 5, 2)
    plt.title("Ground Truth Mask")
    plt.imshow(ground_truth_mask, cmap='gray')
    plt.axis('off')

    # Predicted Mask
    plt.subplot(1, 5, 3)
    plt.title("Predicted Mask")
    plt.imshow(predicted_mask, cmap='gray')
    plt.axis('off')

    # Ground Truth Footprints
    plt.subplot(1, 5, 4)
    plt.title("Ground Truth Footprints")
    plt.imshow(gt_boundary_image, cmap='gray')
    plt.axis('off')

    # Predicted Footprints
    plt.subplot(1, 5, 5)
    plt.title("Predicted Footprints")
    plt.imshow(pred_boundary_image, cmap='gray')
    plt.axis('off')

    plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, jaccard_score

y_true = Y_test.flatten()        
y_pred = (predictions > 0.5).astype(np.uint8).flatten()  

# 1. Mean IoU (Jaccard Index)
iou = jaccard_score(y_true, y_pred, average='micro')
print(f"Mean IoU: {iou:.4f}")

# 2. Accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# 3. Precision and Recall
precision = precision_score(y_true, y_pred, average='binary', pos_label=1)
recall = recall_score(y_true, y_pred, average='binary', pos_label=1)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Assuming `history` is the object returned by `model.fit()`
# Example:
# history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=50, batch_size=4)

# Extract training and validation accuracy from the history object
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Get the number of epochs
epochs = range(1, len(train_accuracy) + 1)

# Plot training and validation accuracy
plt.figure(figsize=(10, 5))

# Plot training accuracy
plt.plot(epochs, train_accuracy, label='Training Accuracy', color='blue')

# Plot validation accuracy
plt.plot(epochs, val_accuracy, label='Validation Accuracy', color='orange')

# Add labels and title
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Show the plot
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, jaccard_score

# Helper function to flatten masks
def flatten_masks(masks):
    """
    Flattens binary masks for evaluation.

    Parameters:
        masks (numpy.ndarray): Binary masks.

    Returns:
        numpy.ndarray: Flattened binary masks.
    """
    return masks.flatten().astype(np.uint8)

# Helper function to calculate metrics
def calculate_metrics(Y_true, Y_pred):
    """
    Calculates evaluation metrics (IoU, Accuracy, Precision, Recall).

    Parameters:
        Y_true (numpy.ndarray): Ground truth masks.
        Y_pred (numpy.ndarray): Predicted masks.

    Returns:
        dict: Dictionary containing IoU, Accuracy, Precision, and Recall.
    """
    # Flatten masks
    gt_flat = flatten_masks(Y_true)
    pred_flat = flatten_masks(Y_pred)

    # Calculate metrics
    iou = jaccard_score(gt_flat, pred_flat, average='binary')
    accuracy = accuracy_score(gt_flat, pred_flat)
    precision = precision_score(gt_flat, pred_flat, average='binary', pos_label=1)
    recall = recall_score(gt_flat, pred_flat, average='binary', pos_label=1)

    return {
        "IoU": iou,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall
    }

# Generate predictions for the test set
predictions_test = model.predict(X_test)
predictions_test = (predictions_test > 0.5).astype(np.uint8)  # Threshold predictions

# Calculate metrics for the test set
test_metrics = calculate_metrics(Y_test, predictions_test)

# Print test metrics
print("\nTest Metrics:")
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.4f}")

# Extract validation data from training data (based on validation_split=0.2)
val_split = 0.2
num_train_samples = len(X_train)
num_val_samples = int(val_split * num_train_samples)

# Split into training and validation sets
X_train_final = X_train[:-num_val_samples]
Y_train_final = Y_train[:-num_val_samples]
X_val = X_train[-num_val_samples:]
Y_val = Y_train[-num_val_samples:]

# Generate predictions for the validation set
predictions_val = model.predict(X_val)
predictions_val = (predictions_val > 0.5).astype(np.uint8)  # Threshold predictions

# Calculate metrics for the validation set
val_metrics = calculate_metrics(Y_val, predictions_val)

# Print validation metrics
print("\nValidation Metrics:")
for metric, value in val_metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Plot only the first 50 epochs
epochs_to_plot = 50

plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'][:epochs_to_plot], label='Training Accuracy', color='blue')
plt.plot(history.history['val_accuracy'][:epochs_to_plot], label='Validation Accuracy', color='orange')
plt.title('Training and Validation Accuracy ')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Extract loss and accuracy
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(loss) + 1)

# Plot loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, 'b-', label='Training Loss')
plt.plot(epochs, val_loss, 'orange', label='Validation Loss')
plt.title('Loss vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, acc, 'b-', label='Training Accuracy')
plt.plot(epochs, val_acc, 'orange', label='Validation Accuracy')
plt.title('Accuracy vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix


labels = ['Background', 'Building']

# Confusion matrices
cm = confusion_matrix(y_true, y_pred)
cm_norm = confusion_matrix(y_true, y_pred, normalize='true')

# Plot side by side
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Raw Confusion Matrix
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=labels, yticklabels=labels, ax=axes[0])
axes[0].set_title('Confusion Matrix (Raw)')
axes[0].set_xlabel('Predicted label')
axes[0].set_ylabel('True label')

# Normalized Confusion Matrix
sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Greens', xticklabels=labels, yticklabels=labels, ax=axes[1])
axes[1].set_title('Normalized Confusion Matrix')
axes[1].set_xlabel('Predicted label')
axes[1].set_ylabel('True label')

plt.tight_layout()
plt.show()
